In [1]:
import spacy, os
nlp = spacy.load('en')

In [2]:
# switch for debug
flag_print = True

# switch to clear existing data
flag_clear = True

#threshold value for determining section
threshold = 0.5

In [14]:
# to get extract sections from the resume -- add or remove from  'similar_to' accordingly
similar_to = {
    'edu' : ['education', 'study', 'academics'],
    'exp' : ['job', 'internship', 'training', 'research', 'carrer', 'profession', 
             'project', 'responsibility', 'description'],
    'skill' : ['skill', 'languages', 'technology', 'frameworks', 'tools'],
    'extra' : ['introduction', 'intro', 'achievement', 'hobby', 'links', 'additional', 'personal']
}

list_of_sections = similar_to.keys()

# to bring similar_words to their normal forms
for section in list_of_sections:
    new_list = []
    
    for word in similar_to[section]:
        docx = nlp(unicode(word))
        new_list.append(docx[0].lemma_)
        
    if flag_print:
        print section, new_list
        
    similar_to[section] = new_list

skill [u'skill', u'language', u'technology', u'framework', u'tool']
extra [u'introduction', u'intro', u'achievement', u'hobby', u'link', u'additional', u'personal']
exp [u'job', u'internship', u'training', u'research', u'carrer', u'profession', u'project', u'responsibility', u'description']
edu [u'education', u'study', u'academic']


In [4]:
# function to return the words in a uniform 
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(unicode(mod_word))

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    
if flag_print:
    test_words = ['Hello!!', '.,<>', 'India', 'of', '..freedoM..', 'e-mail']
    
    for word in test_words:
        print word, '--returned-->', modify(word)

Hello!! --returned--> hello
.,<> --returned--> None
India --returned--> india
of --returned--> None
..freedoM.. --returned--> freedom
e-mail --returned--> email


In [24]:
# utility function to skip line when no alphabet present
def is_bad(line):
    for c in line:
        if (c.isalpha()):
            return False
    return True
      
if flag_print:
    test_words = ['.', '<.>', 'Speak', 'out', '"Eric"', 'freemail...']
    
    for word in test_words:
        print word, '--returned-->', is_bad(word)  

. --returned--> True
<.> --returned--> True
Speak --returned--> False
out --returned--> False
"Eric" --returned--> False
freemail... --returned--> False


In [32]:
if flag_clear:
    for file_name in os.listdir(os.getcwd()+'/prc_data'):
        os.remove(os.getcwd()+'/prc_data/'+file_name)

for file_name in os.listdir(os.getcwd()+'/raw_data'):
    if flag_print:
        print '\n'
        print '*'*25
        print file_name
        print '*'*25
        
    main_file_handler = open('raw_data/'+file_name, 'r')    
    previous_section  = 'extra'
    
    for line in main_file_handler:
        # skip line if empty
        if (len(line.strip()) == 0 or is_bad(line)):
            continue
                
        # processing next line
        list_of_words_in_line = line.split()
        list_of_imp_words_in_line  = []
        
        for i in range(len(list_of_words_in_line)):
            modified_word = modify(list_of_words_in_line[i])
            
            if (modified_word):
                list_of_imp_words_in_line.append(modified_word)

        if (True):
            curr_line = ' '.join(list_of_imp_words_in_line)
            doc = nlp(unicode(curr_line))
            section_value = {}
            
            # initializing section values to zero
            for section in list_of_sections:
                section_value[section] = 0.0
            section_value[None] = 0.0
            
            # updating section values    
            for token in doc:
                for section in list_of_sections:
                    for word in similar_to[section]:
                        word_token = doc.vocab[unicode(word)]
                        section_value[section] = max(section_value[section], float(word_token.similarity(token)))

            # determining the next section based on section values and threshold
            most_likely_section = None
            for section in list_of_sections:
                #print '>>', section, section_value[section]
                if (section_value[most_likely_section] < section_value[section] and section_value[section] > threshold):
                    most_likely_section = section
            
            # updating the section
            if (previous_section != most_likely_section and most_likely_section is not None):
                previous_section = most_likely_section
                

        # writing data to the separate files
        temp_file_handler = open('prc_data/'+file_name[:-4]+'_'+previous_section+'.txt', 'a')
        try:
            docx = nlp(unicode(line))
        except:
            continue  # to handle the odd case of characters like 'x02', etc.
        mod_line = ''
        for token in docx:
            if (not token.is_stop):
                mod_line += token.lemma_ + ' '
        temp_file_handler.write(mod_line)
        temp_file_handler.close()
        
        if flag_print:
            print previous_section, '\t**', mod_line
        
    main_file_handler.close()
        



*************************
cv2.txt
*************************
extra 	** puneet singh 
 
extra 	** new delhi , delhi 
 
skill 	** work professional industry professional work drive environment utilize apply knowledge , skill ability beneficial mutual growth . 
 
extra 	** personal trait 
 
exp 	** professional qualification 
 
exp 	** work experience 
 
skill 	** software developer 
 
skill 	** software - noida , uttar pradesh 
 
skill 	** fresh 
 
skill 	** skill 
 
exp 	** project 
 
edu 	** team size : 1 
 
exp 	** role : developer 
 
skill 	** technology require : .net end , sql server end 
 
exp 	** project manage medicine inventory system medical store system . 
 
exp 	** team size : 2 
 
exp 	** role : developer 
 
skill 	** technology require : c # end , sql server end 
 
exp 	** project mainly allahabad medicos , allahabad . 
 
skill 	** team size : 2 
 
exp 	** role : developer 
 
skill 	** technology require : .net end , sql server end 
 
extra 	** additional information 
 



skill 	** : c , c++ , python ( networkx library ) , parallel programming ( mpi library ) , graph visualizations ( graphviz librar 
 
skill 	** y ) . 
 
skill 	** software : 
 
skill 	** matlab , gnuplot , latex , socnetv , pajek . 
 
exp 	** projects : 
 
exp 	** human activity detection sunspots 
 
exp 	** sql injection attack preventive measure 
 
exp 	** online multiplayer ping 
 
exp 	** pong game c opengl 
 
exp 	** painterly rendering real image imag 
 
skill 	** e processing technique 
 
skill 	** face recognition pca 
 
extra 	** web base online location travel facilitation service 
 
skill 	** android application 
 
edu 	** software allocate seat student examination center 
 
exp 	** industrial training : 
 
exp 	** title : 
 
extra 	** client query registry 
 
extra 	** desc 
 
extra 	** ription : 
 
exp 	** work       oracle   web   support   team     tcs ,   kolkata .   developed     repository     client   information ,     query     
 
skill 	** solution oracle databases 

edu 	** represent iit kanpur 2 student . school target bring outstanding researcher 
 
exp 	** academia , industry & student discuss current research future trend concurrent system design 
 
skill 	** implementation . 
 
skill 	** iwds : second international workshop distributed systems 
 
skill 	** 27 - 29 november 2010 
 
exp 	** description 
 
exp 	** iit kanpur 
 
exp 	** abstract thesis work accept presentation workshop .   present current 
 
exp 	** result & research work researcher student work area . 
 
exp 	** research experience 
 
skill 	** flow - base programming language multi - core architecture 
 
skill 	** january 2010 - till date 
 
exp 	** masters thesis ( work progress ) 
 
exp 	** dept .   cse , iitk 
 
exp 	** advisor : 
 
exp 	** dr.   r.k.ghosh & prof .   harish karnick 
 
exp 	** description : 
 
exp 	** thesis project aim design & development new share - memory parallel pro- 
 
skill 	** gramm language current future generation - core machine . 
 
skill 	**   p

exp 	**   batch     40   freshman       esc101   course .      tutor   design   problem       lab   test 
 
extra 	** exam , teach student arrange tutorial clear student doubt .   provide student additional 
 
extra 	** weekly note 
 
exp 	** , cover topic spread duration semester , assist lecture material 
 
skill 	** provide rich , - depth broad learning experience c programming language . 
 
extra 	** scholastic achievement 
 
exp 	** secure india rank 373 joint entrance examination 2006 conduct iit percentile close 
 
exp 	** 99.9 
 
skill 	** award scholarship national talent search examination(2003 ) . 
 
edu 	** secure   rank   1       branch   &           3   student       school   (   20   branch )       isc   xii 
 
edu 	** examination hold 2005 . 
 
edu 	** secure rank 3 student cms mahanagar branch icse x examination hold 2003 . 
 
skill 	** skills 
 
skill 	** programminglanguage 
 
skill 	** : c , c++ , java , haskell , oz , ml , python , vhdl 
 
skill 	** platform 
 
ski